In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tnrange, tqdm_notebook
import gc

In [2]:
sns.set_context('talk')

In [3]:
import skimage.io as io

In [4]:
DEBUG = True

In [5]:
import theano
theano.config.openmp = True

In [6]:
import keras

Using Theano backend.


# Preprocessing

In [7]:
labels = pd.read_csv('finetuning-train/gt.csv')

In [8]:
labels.head()

,filename,class_id
0,0000.png,50
1,0001.png,24
2,0002.png,25
3,0003.png,37
4,0004.png,13


In [9]:
labels.max()

filename    2499.png
class_id          50
dtype: object

In [10]:
labels.min()

filename    0000.png
class_id           1
dtype: object

Растянем/сожмём изображения, пока большая из сторон не станет равна SIDE_SIZE

In [11]:
SIDE_SIZE = 250

и дополним изображение до квадрата, заполняя отсутствующие клетки CVAL

In [12]:
CVAL = 255 // 2

In [13]:
from os import listdir
from os.path import isfile, join
import warnings
import skimage.color
import skimage.transform

In [14]:
def resize(input_dir, output_dir):
    files = sorted([f for f in listdir(input_dir) if isfile(join(input_dir, f))])
    for file in files:
        img = io.imread(join(input_dir, file))
        if img.ndim == 2:
            img = skimage.color.gray2rgb(img)
        factor = min([SIDE_SIZE / img.shape[i] for i in range(2)])
        resized_img = skimage.transform.rescale(img, factor,
                                               mode='constant', cval=CVAL, preserve_range=True)
        square_image = np.zeros((SIDE_SIZE, SIDE_SIZE, 3), dtype=np.float64) + CVAL
        square_image[(SIDE_SIZE - resized_img.shape[0]) // 2 :
                     resized_img.shape[0] + (SIDE_SIZE - resized_img.shape[0]) // 2,
                    (SIDE_SIZE - resized_img.shape[1]) // 2 :
                     resized_img.shape[1] + (SIDE_SIZE - resized_img.shape[1]) // 2] = resized_img
        resized_img = square_image.astype(np.uint8)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            io.imsave(join(output_dir, file), resized_img)

In [16]:
%%time
resize('finetuning-train/images/', 'finetuning-train/resized250/')

CPU times: user 2min 30s, sys: 5min, total: 7min 30s
Wall time: 3min 22s


In [17]:
%%time
resize('finetuning-test/images/', 'finetuning-test/resized250/')

CPU times: user 31.3 s, sys: 1min, total: 1min 31s
Wall time: 38 s


Раскидаем train по папочкам-классам

In [15]:
import os

In [16]:
def make_dirs(work_dir):
    for index in range(1, 51):
        os.mkdir(join(work_dir, "{:02d}".format(index)))

In [20]:
make_dirs('finetuning-train/resized250')

In [17]:
import shutil

In [22]:
for row in labels.itertuples():
    shutil.move(join('finetuning-train/resized250', row.filename), 
                join('finetuning-train/resized250', "{:02d}".format(row.class_id), row.filename))

Теперь организуем такую же по структуре папку validation, и переместим туда по 10 случайных фотографий из каждого класса

In [23]:
make_dirs('finetuning-train/validation250')

In [18]:
import random

def make_validation(input_dir, output_dir):
    files = random.sample([f for f in listdir(input_dir) if isfile(join(input_dir, f))], 10)
    for file in files:
        shutil.move(join(input_dir, file), join(output_dir, file))

In [25]:
for index in range(1, 51):
    make_validation(join('finetuning-train/resized250', "{:02d}".format(index)),
                    join('finetuning-train/validation250', "{:02d}".format(index)))

Нам понадобятся DataGenerator'ы для обучения и валидации.

In [19]:
from keras.preprocessing.image import ImageDataGenerator

In [20]:
data_gen_args = dict(rotation_range=20.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2,
                     fill_mode='constant',
                     cval=CVAL,
                     horizontal_flip=True,
                     vertical_flip=False,)

Напишем обёртку, которая будет вызывать prepocess_input для картинок из генератора

In [21]:
from keras.applications.resnet50 import preprocess_input

def preprocessed_datagen(datagen):
    for x, y in datagen:
        yield preprocess_input(x), y

In [22]:
batch_size=100

train_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
            'finetuning-train/resized250', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
             batch_size=batch_size, shuffle=True, seed=10))

validation_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
                 'finetuning-train/validation250', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
                 batch_size=batch_size, shuffle=True, seed=10))

Found 2000 images belonging to 50 classes.
Found 500 images belonging to 50 classes.


# Model

Будем использовать ResNet50. Для начала снимем полносвязные слои

In [23]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input

base_model = ResNet50(weights='imagenet', include_top=False, 
                         input_tensor=Input(shape=(SIDE_SIZE, SIDE_SIZE, 3)))

(Subtensor{int64}.0, Elemwise{add,no_inplace}.0, Elemwise{add,no_inplace}.0, Subtensor{int64}.0)


In [24]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 zeropadding2d_1
2 conv1
3 bn_conv1
4 activation_1
5 maxpooling2d_1
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 merge_1
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 merge_2
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 merge_3
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 merge_4
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c
58 merge_5
59 activation_16
60 res3c_branch2a
61 bn3c_branch

Добавим два полносвязных слоя

In [25]:
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.layers import Flatten, GlobalAveragePooling2D

In [26]:
lr=5 * 10 ** -5
decay=10 ** -7
w_reg=5 * 10 ** -5

In [27]:
gc.collect()

966

In [53]:
x = base_model.output

#x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(1024, activation='relu', W_regularizer=l2(w_reg))(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu', W_regularizer=l2(w_reg))(x)
x = Dropout(0.2)(x)

predictions = Dense(50, activation='softmax', W_regularizer=l2(w_reg))(x)

model = Model(input=base_model.input, output=predictions)

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(#optimizer=SGD(lr=lr, decay=decay, nesterov=True, momentum=0.9),
              optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy', 'categorical_crossentropy'])

# Learning

In [54]:
%%time

hist = model.fit_generator(
        train_gen,
        samples_per_epoch=3000,
        nb_epoch=5,
        validation_data=validation_gen,
        nb_val_samples=600,
        )

Epoch 1/5
3000/3000 [==============================] - 3816s - loss: 3.5402 - categorical_accuracy: 0.1827 - categorical_crossentropy: 3.4166 - val_loss: 2.0414 - val_categorical_accuracy: 0.3917 - val_categorical_crossentropy: 2.0414
Epoch 2/5
3000/3000 [==============================] - 3882s - loss: 1.9790 - categorical_accuracy: 0.4383 - categorical_crossentropy: 1.8599 - val_loss: 1.4818 - val_categorical_accuracy: 0.5350 - val_categorical_crossentropy: 1.4818
Epoch 3/5
3000/3000 [==============================] - 3757s - loss: 1.4290 - categorical_accuracy: 0.5930 - categorical_crossentropy: 1.3203 - val_loss: 1.3511 - val_categorical_accuracy: 0.5900 - val_categorical_crossentropy: 1.3511
Epoch 4/5
3000/3000 [==============================] - 3749s - loss: 1.1729 - categorical_accuracy: 0.6557 - categorical_crossentropy: 1.0797 - val_loss: 1.4466 - val_categorical_accuracy: 0.5617 - val_categorical_crossentropy: 1.4466
Epoch 5/5
3000/3000 [==============================] - 3743s

In [55]:
model.save('5epoch_top_resnet.h5')

In [28]:
from keras.models import load_model

In [29]:
model = load_model('5epoch_top_resnet.h5')

(Subtensor{int64}.0, Elemwise{add,no_inplace}.0, Elemwise{add,no_inplace}.0, Subtensor{int64}.0)


Дообучим некоторые последние res-блоки

Сначала разморозим эти блоки

In [30]:
for layer in model.layers[:142]:
   layer.trainable = False
for layer in model.layers[142:]:
   layer.trainable = True

Возьмём RMSprop с меньшим learning rate (по умолчанию lr=$10^{-3}$)

In [31]:
gc.collect()

203

In [32]:
from keras.optimizers import RMSprop

In [33]:
lr=1 * 10 ** -4

In [34]:
model.compile(optimizer=RMSprop(lr=lr),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy', 'categorical_crossentropy'])

В data-генераторах поменяем seed

In [35]:
batch_size=100

train_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
            'finetuning-train/resized250', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
             batch_size=batch_size, shuffle=True, seed=42))

validation_gen = preprocessed_datagen(ImageDataGenerator(**data_gen_args).flow_from_directory(
                 'finetuning-train/validation250', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode='categorical',
                 batch_size=batch_size, shuffle=True, seed=42))

Found 2000 images belonging to 50 classes.
Found 500 images belonging to 50 classes.


In [36]:
%%time

hist = model.fit_generator(
        train_gen,
        samples_per_epoch=3000,
        nb_epoch=2,
        validation_data=validation_gen,
        nb_val_samples=600,
        )

Epoch 1/2
3000/3000 [==============================] - 4599s - loss: 0.5397 - categorical_accuracy: 0.8453 - categorical_crossentropy: 0.4632 - val_loss: 1.0111 - val_categorical_accuracy: 0.7200 - val_categorical_crossentropy: 1.0111
Epoch 2/2
3000/3000 [==============================] - 4770s - loss: 0.3335 - categorical_accuracy: 0.9173 - categorical_crossentropy: 0.2590 - val_loss: 1.0940 - val_categorical_accuracy: 0.7200 - val_categorical_crossentropy: 1.0940
CPU times: user 5h 46min 26s, sys: 57min 11s, total: 6h 43min 37s
Wall time: 2h 39min 10s


In [37]:
model.save('2epoch_top_res_block_resnet.h5')

# Result

Реализуем datagen для тестовой выборки

In [38]:
def test_preprocessed_datagen(datagen):
    for x in datagen:
        yield preprocess_input(x)

In [50]:
batch_size=100

test_gen = test_preprocessed_datagen(ImageDataGenerator().flow_from_directory(
            'finetuning-test/resized250/', target_size=(SIDE_SIZE, SIDE_SIZE), class_mode=None,
             batch_size=batch_size, shuffle=False, seed=10))

Found 500 images belonging to 1 classes.


Получим вероятности классов

In [51]:
TEST_SAMPLES_COUNT = 500
probs = model.predict_generator(test_gen, val_samples=TEST_SAMPLES_COUNT)

Преобразуем вероятности в предсказания

In [52]:
predictions = np.argmax(probs, axis=1) + 1

Сохраним предсказания в файл

In [54]:
df_preds = pd.DataFrame()
df_preds['filename'] = ["{:04d}.png".format(index) for index in range(TEST_SAMPLES_COUNT)]
df_preds['class_id'] = predictions

df_preds.to_csv("csv/dense5_topres2.csv", index=False)